<a href="https://colab.research.google.com/github/vergil-sparda0701/SasakiHaremExtractor/blob/main/SasakiHaremExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DISCLAIMER:

Este proyecto no fue creado por mi, solome encargue de pasar el codigo al entorno de google colab, todos los derechos de creacion del codigo son para **KojiroSfw**

📘 SasakiHaremExtractor
===============================

Este proyecto permite extraer y agrupar automáticamente personajes de estilo anime desde videos,
usando detección facial, filtrado por calidad (nitidez), y agrupamiento visual con CLIP.


🎚️ Controles importantes en la interfaz:
---------------------------------------
- FPS a procesar: reduce la cantidad de frames por segundo a analizar (mayor = más preciso, menor = más rápido).
- Umbral de borrosidad: filtra imágenes borrosas. Mayor valor = más estricto.
- Expansión del recorte: controla cuánto se expande el área alrededor del rostro detectado.
- Similitud: define cuán parecidos deben ser los personajes para agruparlos.

🛑 Puedes detener el proceso con el botón "⏹️ Detener".

📦 Resultados:
--------------
Las imágenes agrupadas se guardarán en carpetas dentro de `out/`, una por cada personaje detectado.

🔧 Soporte:
-----------
Cualquier problema de instalación puede deberse a versiones incompatibles de Python o falta de conexión a internet al instalar dependencias.



In [ ]:
# @title Montar Google Drive
# @markdown Si tienes tus videos en una carpeta en tu Google Drive ejecuta esta celda

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title 🎌 Ejecutar SasakiHaremExtractor
# @markdown Sube videos individuales o selecciona una carpeta completa.

!pip install openai-clip

import os
import cv2
import numpy as np
import torch
import clip
import gradio as gr
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import shutil
import threading
import requests

# Función para descargar el archivo lbpcascade_animeface.xml
def download_cascade_file(filename="lbpcascade_animeface.xml"):
    url = "https://raw.githubusercontent.com/nagadomi/lbpcascade_animeface/master/lbpcascade_animeface.xml"
    if not os.path.exists(filename):
        print(f"Descargando {filename}...")
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            with open(filename, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"{filename} descargado correctamente.")
        except requests.exceptions.RequestException as e:
            print(f"Error al descargar {filename}: {e}")
            return False
    return True

# Asegúrate de que el archivo cascade esté presente
if not download_cascade_file():
    raise FileNotFoundError("No se pudo descargar lbpcascade_animeface.xml. Por favor, inténtalo de nuevo o descárgalo manualmente.")

# Función para abrir carpeta out (adaptada para Colab)
def abrir_carpeta_out():
    out_path = os.path.abspath("out")
    if os.path.exists(out_path):
        # En Google Colab, no podemos abrir una carpeta directamente en el sistema del usuario.
        # En su lugar, proporcionamos instrucciones para descargarla.
        return f"📂 La carpeta 'out/' está lista. Puedes descargarla desde el explorador de archivos de Colab (icono de carpeta a la izquierda)."
    else:
        return "❌ La carpeta 'out/' no existe aún."

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
CASCADE_PATH = 'lbpcascade_animeface.xml'
face_cascade = cv2.CascadeClassifier(CASCADE_PATH)
model, preprocess = clip.load("ViT-B/32", device=DEVICE)
stop_event = threading.Event()

def is_blurry(frame, blur_threshold):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var() < blur_threshold

def find_cluster(embedding, clusters, similarity_threshold):
    if not clusters:
        return None
    sims = cosine_similarity([embedding], [c[0] for c in clusters])[0]
    max_sim = np.max(sims)
    if max_sim > similarity_threshold:
        return np.argmax(sims)
    return None

def stop_processing():
    stop_event.set()
    return "⏹️ Proceso cancelado por el usuario."

def extract_and_group(videos, folder, blur_threshold, crop_margin_percent, similarity_threshold, target_fps):
    stop_event.clear()
    OUTPUT_FOLDER = 'out'
    shutil.rmtree(OUTPUT_FOLDER, ignore_errors=True)
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)

    clusters = []
    all_videos = []

    if folder:
        # En Colab, 'folder' será una ruta de carpeta en el entorno de Colab
        if os.path.isdir(folder):
            all_videos.extend([
                os.path.join(folder, f)
                for f in os.listdir(folder)
                if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))
            ])
        else:
            return f"⚠️ La ruta de carpeta '{folder}' no es válida o no existe."
    elif videos:
        # Gradio en Colab maneja la subida de archivos a rutas temporales
        all_videos.extend([v.name for v in videos]) # Acceder a la ruta del archivo subido

    if not all_videos:
        return "⚠️ No se encontraron videos válidos."

    for video_path in all_videos:
        filename = os.path.basename(video_path)
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"No se pudo abrir el video: {video_path}")
            continue

        video_fps = cap.get(cv2.CAP_PROP_FPS)
        frame_interval = max(1, int(video_fps / target_fps)) if target_fps > 0 else 1
        frame_id = 0

        while cap.isOpened():
            if stop_event.is_set():
                cap.release()
                return "⏹️ Proceso detenido."

            ret, frame = cap.read()
            if not ret:
                break
            frame_id += 1

            if frame_id % frame_interval != 0:
                continue

            if is_blurry(frame, blur_threshold):
                continue

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=3, minSize=(24, 24))

            for (x, y, w, h) in faces:
                margin = int((crop_margin_percent / 100.0) * h)
                x1 = max(x - margin, 0)
                y1 = max(y - margin, 0)
                x2 = min(x + w + margin, frame.shape[1])
                y2 = min(y + h + margin, frame.shape[0])

                face_crop = frame[y1:y2, x1:x2]
                if face_crop.size == 0: # Evitar crops vacíos
                    continue

                image_pil = Image.fromarray(cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB))
                image_input = preprocess(image_pil).unsqueeze(0).to(DEVICE)

                with torch.no_grad():
                    embedding = model.encode_image(image_input).cpu().numpy()[0]

                cluster_idx = find_cluster(embedding, clusters, similarity_threshold)
                if cluster_idx is None:
                    cluster_folder = os.path.join(OUTPUT_FOLDER, f"character_{len(clusters)+1:03}")
                    os.makedirs(cluster_folder, exist_ok=True)
                    clusters.append((embedding, cluster_folder))
                else:
                    cluster_folder = clusters[cluster_idx][1]

                frame_name = f"{os.path.splitext(filename)[0]}_frame{frame_id:06}.jpg"
                image_pil.save(os.path.join(cluster_folder, frame_name))

        cap.release()

    if stop_event.is_set():
        return "⏹️ Proceso cancelado."

    result = "\n".join([os.path.basename(c[1]) for c in clusters])
    return f"✅ Proceso finalizado. Se generaron los siguientes grupos:\n{result}"

with gr.Blocks(title="Extractor Anime") as demo:
    gr.Markdown("## 🎌 Agrupador de Personajes Anime")
    gr.Markdown("Sube videos individuales o selecciona una carpeta completa.")

    with gr.Row():
        # Usar gr.File para la subida de videos
        video_input = gr.File(file_types=[".mp4", ".avi", ".mov", ".mkv"], label="Videos", file_count="multiple")
        # Para la carpeta, el usuario deberá montar Google Drive o subir la carpeta manualmente
        folder_input = gr.Textbox(label="📁 Ruta de carpeta con videos (opcional, ej: /content/my_videos)")

    with gr.Row():
        blur_slider = gr.Slider(10, 300, value=100, step=10, label="Umbral de borrosidad")
        margin_slider = gr.Slider(0, 100, value=30, step=5, label="Expansión del recorte (%)")
        sim_slider = gr.Slider(0.7, 0.99, value=0.85, step=0.01, label="Similitud (CLIP)")
        fps_slider = gr.Slider(1, 30, value=5, step=1, label="FPS a procesar")

    with gr.Row():
        extract_btn = gr.Button("🚀 Procesar")
        stop_btn = gr.Button("⏹️ Detener")
        open_btn = gr.Button("📂 Abrir carpeta de resultados")

    output_box = gr.Textbox(label="📋 Estado / Resultado", lines=10)

    extract_btn.click(
        extract_and_group,
        inputs=[video_input, folder_input, blur_slider, margin_slider, sim_slider, fps_slider],
        outputs=output_box
    )

    stop_btn.click(
        stop_processing,
        inputs=[],
        outputs=output_box
    )

    open_btn.click(
        abrir_carpeta_out,
        inputs=[],
        outputs=output_box
    )

demo.launch(debug=True, share=True) # 'share=True' para obtener un enlace público en Colab

In [ ]:
#@markdown ### ⬇️ CELDA PARA DESCARGAR TODAS LAS IMAGENES DE LA CARPETA OUTPUTS A UN ZIP
import shutil
import os
from google.colab import files

ruta1 = '/content/ReForge/outputs'
ruta2 = '/content/ComfyUI/output'
ruta3 = '/content/out'

if os.path.exists(ruta1):
    ruta_a_descargar = ruta1
elif os.path.exists(ruta2):
    ruta_a_descargar = ruta2
elif os.path.exists(ruta3):
    ruta_a_descargar = ruta3
else:
    raise FileNotFoundError("Ruta no encontrada")

shutil.make_archive('/content/outputs', 'zip', ruta_a_descargar)

files.download('/content/outputs.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>